In [ ]:
import serial
import time

# Function to send AT command and receive response
def send_command(command):
    ser.write((command + '\r\n').encode())
    time.sleep(1)
    response = ser.read(ser.inWaiting()).decode()
    return response.strip()

# Function to read recent SMS messages
def read_recent_sms(count):
    # Send command to select message storage
    response = send_command('AT+CPMS="SM"')
    if "OK" not in response:
        print("Error selecting message storage.")
        return None
    
    # Send command to list received messages
    response = send_command('AT+CMGL="ALL"')
    if "+CMGL: " not in response:
        print("No messages found.")
        return None
    
    # Split response into individual messages
    messages = response.split('+CMGL: ')[1:]
    recent_messages = []
    
    # Extract content of recent messages
    for message in messages[-count:]:
        message_content = message.split('\r\n', 2)[-1].strip()
        recent_messages.append(message_content)
    
    return recent_messages

# Main code
if __name__ == "__main__":
    # Open serial connection to the A9G module
    ser = serial.Serial('COM5', 115200, timeout=1)

    # Wait for the module to initialize
    time.sleep(2)

    try:
        while True:
            # Check if there are any incoming messages
            if ser.inWaiting() > 0:
                # Read the most recent 5 SMS messages
                recent_sms = read_recent_sms(5)
                if recent_sms:
                    print("Recent SMS messages:")
                    for i, message in enumerate(recent_sms, 1):
                        print(f"{i}. {message}")
                else:
                    print("Failed to read recent SMS messages.")
            time.sleep(1)  # Wait for 1 second before checking again

    except KeyboardInterrupt:
        print("Exiting program.")

    # Close serial connection
    ser.close()
